In [5]:
###loading the library

In [11]:
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import float32,int32
from pandas import read_csv
import pandas as pd
import seaborn as sns
from keras import optimizers

from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.utils import np_utils

# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
print('keras version ', keras.__version__)
# Same labels will be reused throughout the program
LABELS = ['Downstairs',
          'Jogging',
          'Sitting',
          'Standing',
          'Upstairs',
          'Walking']
# The number of steps within one time segment
TIME_PERIODS = 80
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 40

def read_data(file_path):

    column_names = ['user-id',
                    'activity',
                    'timestamp',
                    'x-axis',
                    'y-axis',
                    'z-axis']
    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)
    # Last column has a ";" character which must be removed ...
    df['z-axis'].replace(regex=True,
      inplace=True,
      to_replace=r';',
      value=r'')
    # ... and then this column must be transformed to float explicitly
    df['z-axis'] = df['z-axis'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df.dropna(axis=0, how='any', inplace=True)

    return df
"""The dropna() function is used to remove a row or a column from a dataframe which has a NaN or no values in it. Egs : 
The fillna() function is used to fill the the missing or NaN values in the pandas dataframe with a suitable data 
as decided by the user."""
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan
    
def show_basic_dataframe_info(dataframe):

    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))
# Load data set containing all the data from csv
df = read_data('WISDM_ar_v1.1_raw.txt')
# Describe the data
df['mean']=df[['x-axis','y-axis','z-axis']].mean(axis=1)
df['std']=df[['x-axis','y-axis','z-axis']].std(axis=1)
df['median']=df[['x-axis','y-axis','z-axis']].median(axis=1)
df.head(20)
show_basic_dataframe_info(df)
    
# Define column name of the label vector
LABEL = 'ActivityEncoded'
lables=np.array(LABELS)
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df[LABEL] = le.fit_transform(df['activity'].values.ravel())     
# Differentiate between test set and training set
df_test = df[df['user-id'] > 28]
df_train = df[df['user-id'] <= 28]        

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
df_train['x-axis'] = df_train['x-axis'] / df_train['x-axis'].max()
df_train['y-axis'] = df_train['y-axis'] / df_train['y-axis'].max()
df_train['z-axis'] = df_train['z-axis'] / df_train['z-axis'].max()

# Round numbers 
#df_train = df_train.round({'x-axis': 4, 'y-axis': 4, 'z-axis': 4})
                             
# Normalize features for test data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
df_test['x-axis'] = df_test['x-axis'] / df_test['x-axis'].max()
df_test['y-axis'] = df_test['y-axis'] / df_test['y-axis'].max()
df_test['z-axis'] = df_test['z-axis'] / df_test['z-axis'].max()
# Round numbers
#df_test = df_test.round({'x-axis': 4, 'y-axis': 4, 'z-axis': 4})  

def create_segments_and_labels(df, time_steps, step, label_name):

    # x, y, z acceleration as features
    N_FEATURES = 3
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['x-axis'].values[i: i + time_steps]
        ys = df['y-axis'].values[i: i + time_steps]
        zs = df['z-axis'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels


def create_segments_and_labels2(df, time_steps, step, label_name):

    # x, y, z acceleration as features
    N_FEATURES = 6
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['x-axis'].values[i: i + time_steps]
        ys = df['y-axis'].values[i: i + time_steps]
        zs = df['z-axis'].values[i: i + time_steps]
        means = df['mean'].values[i: i + time_steps]
        stdv = df['std'].values[i: i + time_steps]
        medians = df['median'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs, means, stdv, medians])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

def load_dataset(prefix=''):
    x_train, y_train = create_segments_and_labels(df_train,
                                              80,
                                              40,
                                              LABEL)
    x_test, y_test = create_segments_and_labels(df_test,
                                              TIME_PERIODS,
                                              STEP_DISTANCE,
                                              LABEL)

    
    print('x_train shape: ', x_train.shape)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    print('x_test shape: ', x_test.shape)
    print(x_test.shape[0], 'test samples')
    print('y_test: ', y_test.shape)  
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    num_classes = le.classes_.size
    print('number of classes:',num_classes)
    print(list(le.classes_))
    #OUTPUT:
    #x_train shape:  (20868, 80, 3)
    #20868 training samples
    #y_train shape:  (20868,)
    #x_test shape:  (6584, 80, 3)
    #6584 test samples
    #y_test:  (6584,)
    #number of classes: 6

    input_shape = (num_time_periods*num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train.shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    y_train = y_train.astype('float32')
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print('New y_train shape: ', y_train_hot.shape)
    #OUTPUT:
    #x_train shape: (20868, 240)
    #input_shape: 240
    #New y_train shape:  (20868, 6)
                     
    # Set input & output dimensions
    num_time_periods, num_sensors = x_test.shape[1], x_test.shape[2]
    num_classes = le.classes_.size
    print(list(le.classes_))    
    #input_shape1 = (num_time_periods*num_sensors)
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    print('x_test shape:', x_test.shape)
    print('input_shape:', input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    print('New y_test shape: ', y_test_hot.shape)
    #OUTPUT:
    #x_test shape: (6584, 240)
    #input_shape: 240
    #New y_test shape:  (6584, 6)
    return x_train, y_train, x_test, y_test, y_train_hot, y_test_hot, num_classes
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
    
def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

keras version  2.2.4
Number of columns in the dataframe: 9
Number of rows in the dataframe: 1098203



In [55]:
x_train, y_train, x_test, y_test, y_train_hot, y_test_hot, num_classes = load_dataset()
tick_marks = np.arange(num_classes)
# fit and evaluate a model
verbose, epochs, batch_size, validation_split = 1, 30, 32, 0.2
#trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
model = Sequential()
#model.add(Reshape((80, 3), input_shape=(240,)))
model.add(Conv1D(filters=240, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=240, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=240, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters=30, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters=20, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))        
#model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
#model.add(Dropout(0.5))
#model.add(Flatten())
model.add(Bidirectional(LSTM(40)))
model.add(Dense(40, activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(n_outputs, activation='softmax'))
model.add(Dense(6, activation='softmax'))
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(x_train,y_train_hot,batch_size=batch_size,epochs=epochs,validation_split=validation_split,shuffle=False,verbose=verbose)


x_train shape:  (20868, 80, 3)
20868 training samples
y_train shape:  (20868,)
x_test shape:  (6584, 80, 3)
6584 test samples
y_test:  (6584,)
number of classes: 6
['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']
x_train shape: (20868, 80, 3)
input_shape: 240
New y_train shape:  (20868, 6)
['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']
x_test shape: (6584, 80, 3)
input_shape: 240
New y_test shape:  (6584, 6)
Train on 16694 samples, validate on 4174 samples
Epoch 1/30
16694/16694 [==============================] - 82s 5ms/step - loss: 1.4343 - acc: 0.4590 - val_loss: 1.0873 - val_acc: 0.6284
Epoch 2/30
16694/16694 [==============================] - 68s 4ms/step - loss: 1.1883 - acc: 0.5532 - val_loss: 1.0634 - val_acc: 0.6270
Epoch 3/30
16694/16694 [==============================] - 61s 4ms/step - loss: 1.0756 - acc: 0.5942 - val_loss: 0.9564 - val_acc: 0.6687
Epoch 4/30
16694/16694 [==============================] - 61s 4ms/step - loss: 0

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_63 (Conv1D)           (None, 78, 240)           2400      
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 76, 240)           173040    
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 74, 240)           173040    
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 37, 240)           0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 80)                89920     
_________________________________________________________________
dense_55 (Dense)             (None, 40)                3240      
_________________________________________________________________
dense_56 (Dense)             (None, 6)                 246       
Total para

In [86]:
_, accuracy = model.evaluate(x_test, y_test_hot, batch_size=100, verbose=1)
print(accuracy)

6584/6584 [==============================] - 4s 665us/step
0.8651275846604733


In [58]:
# remove the 1st output layer Dense with 6 nodes
model1 = Model(inputs=model.inputs, outputs=model.layers[-2].output)
model2 = Model(inputs=model.inputs, outputs=model.layers[-3].output)
dense_output_train= model1.predict(x_train)
print('dense_output_train: ',dense_output_train)
dense_output_test = model1.predict(x_test)
print('dense_output_test: ',dense_output_test)
lstm_output_train= model2.predict(x_train)
print('lstm_output_train: ',lstm_output_train)
lstm_output_test = model2.predict(x_test)
print('lstm_output_test: ',lstm_output_test)

dense_output_train:  [[0.23987806 0.05347753 0.6474683  ... 0.34367177 1.4511887  0.        ]
 [0.         0.         3.1024737  ... 0.         0.5111427  0.        ]
 [0.         0.         3.1931255  ... 0.         0.3927821  0.        ]
 ...
 [3.3549821  0.         0.18285824 ... 0.90554565 0.4637118  0.        ]
 [3.360399   0.         0.1556958  ... 0.85389495 0.50340396 0.        ]
 [3.3336196  0.         0.12943435 ... 0.80421746 0.5454112  0.        ]]
dense_output_test:  [[0.         2.9544418  0.5974821  ... 0.         1.1299223  0.        ]
 [0.         2.9302182  0.30489492 ... 0.         1.6569209  0.        ]
 [0.         3.5448213  0.42399004 ... 0.         1.3709013  0.        ]
 ...
 [2.7110136  0.         0.         ... 0.         2.2075527  1.0575529 ]
 [2.8651204  0.         0.         ... 0.09580979 1.5513363  0.7193655 ]
 [2.5414934  0.         0.         ... 0.20998378 1.9153293  0.8030261 ]]
lstm_output_train:  [[-0.40384573  0.00175198 -0.4561716  ... -0.530481

In [60]:
lstm_output_train[0]

array([-4.0384573e-01,  1.7519804e-03, -4.5617160e-01, -5.4254198e-01,
       -9.1551447e-01, -5.8552402e-01, -2.2497436e-01,  1.7196024e-02,
        3.3672905e-01, -2.7825966e-01, -0.0000000e+00, -5.3220063e-01,
       -7.7723660e-02,  4.7099569e-01, -5.3261864e-01, -2.4673425e-02,
       -9.2872840e-01,  7.0229161e-01, -1.5075578e-01,  8.2796735e-01,
       -9.0161914e-01,  7.8428030e-01, -5.2981985e-01, -2.2619471e-01,
        3.9849550e-01,  4.6065465e-01,  3.5378963e-01,  0.0000000e+00,
       -8.1180465e-01, -5.6276661e-01, -7.4427718e-01,  4.2225328e-01,
        3.1217813e-01,  2.1755286e-01,  0.0000000e+00,  6.3916230e-01,
        5.6229365e-01,  2.4953611e-01, -1.3542669e-01,  3.4550339e-01,
        7.0415431e-01, -3.1640801e-01,  7.3656070e-01, -1.7475872e-01,
       -7.9246598e-01, -3.4594059e-01,  8.8329577e-01,  2.7796079e-03,
       -5.7709240e-03, -5.9244049e-01, -3.0640058e-02, -7.1249449e-01,
       -2.0003103e-02,  3.9766049e-01, -5.5262931e-02, -5.3872335e-01,
      

In [62]:
np.savetxt('wisdm_dense_output_train.csv',dense_output_train, delimiter=',', fmt='%.7e')
np.savetxt('wisdm_dense_output_test.csv',dense_output_test, delimiter=',', fmt='%.7e')
np.savetxt('wisdm_lstm_output_train.csv',lstm_output_train, delimiter=',', fmt='%.7e')
np.savetxt('wisdm_lstm_output_test.csv',lstm_output_test, delimiter=',', fmt='%.7e')

In [63]:
lstm_output_train[3]

array([-9.9031663e-01,  2.8833112e-01,  1.0000000e+00, -7.6178789e-01,
       -7.3136973e-01, -6.5327728e-01, -0.0000000e+00, -0.0000000e+00,
        5.9522808e-01, -9.4068140e-01, -0.0000000e+00, -2.0836160e-01,
       -0.0000000e+00, -7.7590847e-01,  8.7838304e-01,  0.0000000e+00,
       -6.4994526e-01,  3.4887323e-01, -0.0000000e+00, -9.1350293e-01,
       -4.0666392e-01,  7.4899799e-01, -1.0364174e-01, -0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
       -7.6157671e-01, -7.7936783e-02,  8.5394484e-01,  0.0000000e+00,
        2.2441733e-01, -3.3288472e-03,  0.0000000e+00, -3.9170548e-01,
        4.6815145e-01,  0.0000000e+00,  8.0785602e-02, -3.9510533e-01,
       -2.5866184e-01, -2.1892807e-01,  7.2969264e-01, -1.8718183e-01,
       -1.0000000e+00, -4.8871773e-01,  9.1710889e-01,  1.0223253e-02,
       -3.0697521e-03,  8.8223612e-01, -6.0700792e-08, -2.5128874e-01,
        4.3397808e-01,  5.8786273e-01, -6.5807323e-04, -7.5679213e-02,
      

In [80]:
#Generally, we can use unlabelled data to help initialize large models, like deep neural networks.
#More specifically, we can label unlabelled data, have it corroborate the prediction if needed, 
#and use that as input to update or retrain a model to make be better for future predictions.
output_train=model.predict(x_train)
output_test=model.predict(x_test)


In [124]:
df['activity']

0          Jogging
1          Jogging
2          Jogging
3          Jogging
4          Jogging
            ...   
1098199    Sitting
1098200    Sitting
1098201    Sitting
1098202    Sitting
1098203    Sitting
Name: activity, Length: 1098203, dtype: object

In [119]:
lables

array(['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs',
       'Walking'], dtype='<U10')

In [127]:
y_pred = np.argmax(output_test,axis=1)
print(y_pred)

[1 1 1 ... 3 3 3]


In [128]:
y_pred[0]

1

In [129]:
y_test[0]

1.0

In [67]:
np.savetxt('wisdm_probability_output_train2.csv',output_train, delimiter=',',fmt='%.7e')
np.savetxt('wisdm_probability_output_test2.csv',output_test, delimiter=',',fmt='%.7e')

In [68]:
train_output=np.argmax(output_train,axis=1)
np.savetxt('uci_probability_output_train.csv',train_output, delimiter=',',fmt='%i')
test_output=np.argmax(output_test,axis=1)
np.savetxt('uci_probability_output_test.csv',test_output, delimiter=',',fmt='%i')

In [69]:
train_output

array([5, 5, 5, ..., 2, 2, 2], dtype=int64)

In [131]:
result_train=[]
for i in train_output:
    if i == 0:
        result_train.append('Downstairs')
    if i == 1:
        result_train.append('Jogging')
    if i == 2:
        result_train.append('Sitting')
    if i == 3:
        result_train.append('Standing')
    if i == 4:
        result_train.append('Upstairs')
    if i == 5:
        result_train.append('Walking')

print(result_train)

['Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Upstairs', 'Upstairs', 'Downstairs', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jog

In [132]:
result_test=[]
for i in test_output:
    if i == 0:
        result_test.append('Downstairs')
    if i == 1:
        result_test.append('Jogging')
    if i == 2:
        result_test.append('Sitting')
    if i == 3:
        result_test.append('Standing')
    if i == 4:
        result_test.append('Upstairs')
    if i == 5:
        result_test.append('Walking')
print(result_test)

['Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Upstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Upstairs', 'Upstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Upstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs',

In [133]:
npa_train=np.asarray(result_train)
npa_test=np.asarray(result_test)

In [134]:
np.savetxt('wisdm_probability_output_train_classfication.csv',npa_train, delimiter=',',fmt='%s')
np.savetxt('wisdm_probability_output_test_classfication.csv',npa_test, delimiter=',',fmt='%s')

In [135]:
df_dense_test=pd.DataFrame(dense_output_test)
df_dense_train=pd.DataFrame(dense_output_train)
df_lstm_test=pd.DataFrame(lstm_output_test)
df_lstm_train=pd.DataFrame(lstm_output_train)

df_train=pd.DataFrame(npa_train)
df_test=pd.DataFrame(npa_test)

from sklearn.preprocessing import normalize
normalized_df_dense_test=pd.DataFrame(normalize(df_dense_test))
normalized_df_dense_train=pd.DataFrame(normalize(df_dense_train))
normalized_df_lstm_test=pd.DataFrame(normalize(df_lstm_test))
normalized_df_lstm_train=pd.DataFrame(normalize(df_lstm_train))

In [136]:
df_test

,0
0,Jogging
1,Jogging
2,Jogging
3,Jogging
4,Jogging
...,...
6579,Standing
6580,Sitting
6581,Standing
6582,Standing


In [137]:
merged_dense_test_file=pd.concat([normalized_df_dense_test,df_test],axis=1)
merged_dense_train_file=pd.concat([normalized_df_dense_train,df_train],axis=1)
merged_lstm_test_file=pd.concat([normalized_df_lstm_test,df_test],axis=1)
merged_lstm_train_file=pd.concat([normalized_df_lstm_train,df_train],axis=1)

In [139]:
np.savetxt('wisdm_merged_dense_test_file.csv',merged_dense_test_file, delimiter=',',fmt='%s')
np.savetxt('wisdm_merged_dense_train_file.csv',merged_dense_train_file, delimiter=',',fmt='%s')
np.savetxt('wisdm_merged_lstm_test_file.csv',merged_lstm_test_file, delimiter=',',fmt='%s')
np.savetxt('wisdm_merged_lstm_train_file.csv',merged_lstm_train_file, delimiter=',',fmt='%s')

In [140]:
merged_dense_test_file

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,0
0,0.0,0.4,0.1,0.0,0.0,0.0,0.0,0.0,0.3,0.0,...,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,Jogging
1,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,...,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.2,0.0,Jogging
2,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0,Jogging
3,0.0,0.3,0.1,0.0,0.0,0.0,0.0,0.0,0.4,0.0,...,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.2,0.0,Jogging
4,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,...,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.2,0.0,Jogging
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6579,0.5,0.0,0.0,0.0,0.1,0.0,0.1,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.4,0.2,Standing
6580,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.4,0.0,0.0,...,0.0,0.0,0.0,0.1,0.1,0.5,0.1,0.1,0.0,Sitting
6581,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.4,0.2,Standing
6582,0.5,0.0,0.0,0.0,0.1,0.0,0.1,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.3,0.1,Standing


In [141]:
result_train2=[]
for i in y_train:
    if i == 0:
        result_train2.append('Downstairs')
    if i == 1:
        result_train2.append('Jogging')
    if i == 2:
        result_train2.append('Sitting')
    if i == 3:
        result_train2.append('Standing')
    if i == 4:
        result_train2.append('Upstairs')
    if i == 5:
        result_train2.append('Walking')
print(result_train2)

['Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging'

In [142]:
result_test2=[]
for i in y_test:
    if i == 0:
        result_test2.append('Downstairs')
    if i == 1:
        result_test2.append('Jogging')
    if i == 2:
        result_test2.append('Sitting')
    if i == 3:
        result_test2.append('Standing')
    if i == 4:
        result_test2.append('Upstairs')
    if i == 5:
        result_test2.append('Walking')
print(result_test2)

['Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Jogging', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Walking', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Upstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downstairs', 'Downsta

In [143]:
df_train2=pd.DataFrame(result_train2)
df_test2=pd.DataFrame(result_test2)


In [144]:
normalized_df_dense_test

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.0,0.4,0.1,0.0,0.0,0.0,0.0,0.0,0.3,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0
1,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.2,0.0
2,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0
3,0.0,0.3,0.1,0.0,0.0,0.0,0.0,0.0,0.4,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.2,0.0
4,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6579,0.5,0.0,0.0,0.0,0.1,0.0,0.1,0.4,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.4,0.2
6580,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.4,0.0,0.0,...,0.2,0.0,0.0,0.0,0.1,0.1,0.5,0.1,0.1,0.0
6581,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.4,0.2
6582,0.5,0.0,0.0,0.0,0.1,0.0,0.1,0.4,0.0,0.0,...,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.3,0.1


In [145]:
merged_dense_test_file2=pd.concat([normalized_df_dense_test,df_test2],axis=1)
merged_dense_train_file2=pd.concat([normalized_df_dense_train,df_train2],axis=1)
merged_lstm_test_file2=pd.concat([normalized_df_lstm_test,df_test2],axis=1)
merged_lstm_train_file2=pd.concat([normalized_df_lstm_train,df_train2],axis=1)

In [148]:
merged_dense_train_file2

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,0
0,0.1,0.0,0.1,0.0,0.1,0.1,0.0,0.2,0.0,0.0,...,0.6,0.0,0.2,0.1,0.0,0.0,0.1,0.3,0.0,Walking
1,0.0,0.0,0.3,0.0,0.0,0.2,0.0,0.0,0.1,0.0,...,0.6,0.0,0.1,0.0,0.0,0.3,0.0,0.1,0.0,Walking
2,0.0,0.0,0.3,0.0,0.0,0.2,0.0,0.0,0.2,0.0,...,0.6,0.0,0.1,0.0,0.0,0.3,0.0,0.0,0.0,Walking
3,0.0,0.0,0.4,0.0,0.0,0.2,0.0,0.0,0.3,0.0,...,0.6,0.0,0.1,0.0,0.0,0.3,0.0,0.1,0.0,Walking
4,0.0,0.0,0.4,0.0,0.0,0.1,0.0,0.0,0.2,0.0,...,0.6,0.0,0.1,0.0,0.0,0.3,0.0,0.1,0.0,Walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20863,0.6,0.0,0.0,0.0,0.1,0.0,0.0,0.4,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.5,0.1,0.1,0.0,Sitting
20864,0.5,0.0,0.0,0.0,0.1,0.0,0.0,0.4,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.5,0.1,0.1,0.0,Sitting
20865,0.5,0.0,0.0,0.0,0.1,0.0,0.0,0.4,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.5,0.1,0.1,0.0,Sitting
20866,0.5,0.0,0.0,0.0,0.1,0.0,0.0,0.4,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.5,0.1,0.1,0.0,Sitting


In [149]:
np.savetxt('wisdm_merged_dense_test_file2.csv',merged_dense_test_file2, delimiter=',',fmt='%s')
np.savetxt('wisdm_merged_dense_train_file2.csv',merged_dense_train_file2, delimiter=',',fmt='%s')
np.savetxt('wisdm_merged_lstm_test_file2.csv',merged_lstm_test_file2, delimiter=',',fmt='%s')
np.savetxt('wisdm_merged_lstm_train_file2.csv',merged_lstm_train_file2, delimiter=',',fmt='%s')